<a href="https://colab.research.google.com/github/mardenlelis/CalculadoraIlimitada/blob/main/AorticValve_Segmentation_Pipeline_COMPLETE_Testada_Repetida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🫀 Aortic Valve Segmentation Pipeline with nnU-Net

This notebook contains the full pipeline from patch extraction to model training for aortic valve segmentation in non-contrast CT scans.

---

In [ ]:
##Passo 01 Para começar o algoritmo, é aqui: Montar o Google Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 🔧 Install Dependencies

In [ ]:
##Passo 2 - sempre que reiniciar, tem que rodar tudo de novo!
!pip install --upgrade pip
!pip install numpy pandas nibabel tqdm SimpleITK openpyxl
!pip install nibabel
!pip install medpy
!pip install nnunetv2
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 108.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'medpy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'medpy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=8133798620079592e733c33dffedfdc7e3bd2a303a18d3efe1aabf3ccbce6490
  Stored in directory: /root/.cache/pip/wheels/d4/33/ed/

In [ ]:
#Para ter certeza que está usando GPU
import torch
print("CUDA disponível:", torch.cuda.is_available())
print("Dispositivo:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA disponível: True
Dispositivo: Tesla T4


In [ ]:
#Precisa mostrar as pastas que a nnUNet vai trabalhar
import os

os.environ['nnUNet_raw'] = '/content/drive/MyDrive/nnunet_data/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/content/drive/MyDrive/nnunet_data/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/content/drive/MyDrive/nnunet_data/nnUNet_results'

# Para checar:
print("nnUNet_raw:", os.environ['nnUNet_raw'])
print("nnUNet_preprocessed:", os.environ['nnUNet_preprocessed'])
print("nnUNet_results:", os.environ['nnUNet_results'])


nnUNet_raw: /content/drive/MyDrive/nnunet_data/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnunet_data/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnunet_data/nnUNet_results


In [ ]:
# Apaga todos os arquivos dentro das pastas Images e Labels para começar do zero
#!rm -f /content/drive/MyDrive/nnUNet/Images/*
#!rm -f /content/drive/MyDrive/nnUNet/Labels/*
!rm -f /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/imagens/*
!rm -f /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/rotulos/*


#Diretório garantido: '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/imagens'
#Diretório garantido: '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/rotulos'
print("Pastas 'Images' e 'Labels' limpas com sucesso!")

In [ ]:
#Essa célula copia os arquivos do diretório origem e transfere para a estrutura de arquivos utilizadas pelas células de preparação do dataset

import os
import shutil

# --- Caminhos ---
dir_origem = '/content/drive/MyDrive/TC_DICOM/nrrd'
#dir_destino_imagens = '/content/drive/MyDrive/nnUNet/Images'
#dir_destino_labels = '/content/drive/MyDrive/nnUNet/Labels'

# Diretórios de destino para as imagens e os rótulos (labels)
dir_destino_imagens = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/imagens'
dir_destino_labels = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/originais/rotulos'

# Diretórios adicionais que precisam ser validados/criados
#dir_patches_imagens = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/imagesTr'
#dir_patches_labels = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/labelsTr'

print("Iniciando a busca, cópia e RENOMEAÇÃO com critérios específicos...")

if not os.path.isdir(dir_origem):
    print(f"\nERRO: O diretório de origem '{dir_origem}' não foi encontrado.")
else:
    for raiz, subpastas, arquivos in os.walk(dir_origem):
        if raiz == dir_origem:
            continue

        id_paciente_str = os.path.basename(raiz)
        if not id_paciente_str.isdigit():
            continue

        id_paciente_formatado = id_paciente_str.zfill(3)
        print(f"\nAnalisando pasta do paciente: {id_paciente_formatado}")

        # Variáveis para garantir que apenas um arquivo de cada tipo seja copiado
        label_copiado = False
        imagem_copiada = False

        for nome_arquivo in arquivos:
            caminho_completo_origem = os.path.join(raiz, nome_arquivo)

            # --- LÓGICA ATUALIZADA E MAIS ESPECÍFICA ---

            # 1. Procura pelo arquivo de rótulo que termina com um padrão específico
            #    Isso evita capturar outros arquivos como '-label_1.nrrd' ou '.ctbl'
            if nome_arquivo.endswith('.seg.nrrd') and not label_copiado:
                novo_nome_label = f"{id_paciente_formatado}.nrrd"
                caminho_completo_destino = os.path.join(dir_destino_labels, novo_nome_label)
                print(f"  -> RÓTULO encontrado e renomeado para: {novo_nome_label}")
                shutil.copy2(caminho_completo_origem, caminho_completo_destino)
                label_copiado = True # Marca como copiado para não sobrescrever

            # 2. Procura pelo arquivo de imagem com nome exato
            if nome_arquivo.endswith('120 KV.nrrd') and not imagem_copiada:
                novo_nome_imagem = f"{id_paciente_formatado}.nrrd"
                caminho_completo_destino = os.path.join(dir_destino_imagens, novo_nome_imagem)
                print(f"  -> IMAGEM encontrada e renomeada para: {novo_nome_imagem}")
                shutil.copy2(caminho_completo_origem, caminho_completo_destino)
                imagem_copiada = True # Marca como copiado

        # Avisa se algum arquivo não foi encontrado na pasta do paciente
        if not label_copiado:
            print(f"  -> AVISO: Nenhum arquivo de RÓTULO correspondente a '*-label.nrrd' foi encontrado.")
        if not imagem_copiada:
            print(f"  -> AVISO: Nenhum arquivo de IMAGEM '2 120 KV.nrrd' foi encontrado.")


    print("\n\nProcesso de cópia e renomeação concluído!")

In [ ]:
# Instala a biblioteca necessária para manipulação de imagens médicas
!pip install -q SimpleITK

import os
import shutil
import json
import SimpleITK as sitk # Biblioteca para ler e salvar imagens médicas corretamente

# ===================================================================
## 1. CONFIGURAÇÃO DE DIRETÓRIOS E LIMPEZA
# ===================================================================

# Nome do dataset, usado como prefixo para os novos arquivos
DATASET_NAME = "AorticValve"

# Diretório base onde a estrutura do nnU-Net será criada
BASE_DIR = '/content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/'

# !!! DIRETÓRIOS DE ORIGEM MANTIDOS CONFORME O SEU SCRIPT ORIGINAL !!!
SOURCE_IMAGES_DIR = os.path.join(BASE_DIR, "originais", "imagens")
SOURCE_LABELS_DIR = os.path.join(BASE_DIR, "originais", "rotulos")

# Define os diretórios de destino conforme a estrutura do nnU-Net
IMAGES_TR_DIR = os.path.join(BASE_DIR, "imagesTr")
LABELS_TR_DIR = os.path.join(BASE_DIR, "labelsTr")
IMAGES_TS_DIR = os.path.join(BASE_DIR, "imagesTs")

# --- LIMPEZA DOS DIRETÓRIOS DE DESTINO ---
print("🧹 Limpando diretórios de destino para um novo processamento...")
for path in [IMAGES_TR_DIR, LABELS_TR_DIR, IMAGES_TS_DIR]:
    # Cria o diretório se ele não existir
    os.makedirs(path, exist_ok=True)
    # Apaga todos os arquivos dentro do diretório para garantir que não haja lixo de execuções anteriores
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)

# ===================================================================
## 2. FUNÇÃO DE CONVERSÃO E CÓPIA CORRETA PARA .gz
# ===================================================================

def convert_and_save_gzipped(source_path, dest_path):
    """
    Lê um arquivo de imagem (ex: .nii) e o salva no formato .nii.gz,
    garantindo que a compressão gzip seja aplicada corretamente.
    """
    if not os.path.exists(source_path):
        print(f"  -> ⚠️ Arquivo de origem não encontrado: {os.path.basename(source_path)}")
        return False
    try:
        image = sitk.ReadImage(source_path)
        sitk.WriteImage(image, dest_path, useCompression=True) # Força a compressão
        return True
    except Exception as e:
        print(f"  -> ❌ Erro ao processar {os.path.basename(source_path)}: {e}")
        return False

# ===================================================================
## 3. ORGANIZAÇÃO E CONVERSÃO DOS ARQUIVOS
# ===================================================================

print("\n🔎 Listando e processando arquivos de origem...")
image_files = sorted([f for f in os.listdir(SOURCE_IMAGES_DIR) if not f.startswith('.')])
label_files = sorted([f for f in os.listdir(SOURCE_LABELS_DIR) if not f.startswith('.')])

num_train_files = 0
num_test_files = 0

# --- Divisão Treino (80 amostras) ---
print(f"✈️  Processando e convertendo {min(80, len(image_files))} arquivos de treino...")
for i in range(min(80, len(image_files))):
    original_image_path = os.path.join(SOURCE_IMAGES_DIR, image_files[i])
    original_label_path = os.path.join(SOURCE_LABELS_DIR, label_files[i])

    new_image_name = f"{DATASET_NAME}_{i+1:03d}_0000.nii.gz"
    new_label_name = f"{DATASET_NAME}_{i+1:03d}.nii.gz"

    # Converte e salva a imagem e o rótulo
    img_success = convert_and_save_gzipped(original_image_path, os.path.join(IMAGES_TR_DIR, new_image_name))
    lbl_success = convert_and_save_gzipped(original_label_path, os.path.join(LABELS_TR_DIR, new_label_name))

    if img_success and lbl_success:
        num_train_files += 1

# --- Divisão Teste (próximas 20 amostras) ---
print(f"\n🧪 Processando e convertendo {min(20, max(0, len(image_files) - 80))} arquivos de teste...")
for i in range(80, min(100, len(image_files))):
    original_image_path = os.path.join(SOURCE_IMAGES_DIR, image_files[i])
    new_image_name = f"{DATASET_NAME}_{i+1:03d}_0000.nii.gz"

    if convert_and_save_gzipped(original_image_path, os.path.join(IMAGES_TS_DIR, new_image_name)):
        num_test_files += 1

# ===================================================================
## 4. CRIAÇÃO DO ARQUIVO `dataset.json` (FORMATO nnU-Net v2)
# ===================================================================

print("\n📝 Gerando o arquivo dataset.json...")

# Estrutura limpa e correta para nnU-Net v2 para evitar erros no pré-processamento
dataset_info = {
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "background": "0",
        "aortic_valve": "1"
    },
    "numTraining": num_train_files,
    "file_ending": ".nii.gz"
}

# Salva o dicionário como um arquivo JSON formatado
json_path = os.path.join(BASE_DIR, "dataset.json")
with open(json_path, "w", encoding='utf-8') as f:
    json.dump(dataset_info, f, indent=4)

print("\n✅ Processo concluído! Dataset organizado e convertido com sucesso.")
print(f"Total de arquivos de treino processados: {num_train_files}")
print(f"Total de arquivos de teste processados: {num_test_files}")

🧹 Limpando diretórios de destino para um novo processamento...

🔎 Listando e processando arquivos de origem...
✈️  Processando e convertendo 80 arquivos de treino...

🧪 Processando e convertendo 20 arquivos de teste...

📝 Gerando o arquivo dataset.json...

✅ Processo concluído! Dataset organizado e convertido com sucesso.
Total de arquivos de treino processados: 80
Total de arquivos de teste processados: 20


## 🗂️ Prepare nnU-Net Dataset Structure

## 🔄 Run Dataset Planning and Preprocessing

In [ ]:
!nnUNetv2_plan_and_preprocess -d 1 -c 3d_fullres --verify_dataset_integrity --clean -np 1


Fingerprint extraction...
Dataset001_AorticValve
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 80/80 [00:13<00:00,  5.95it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.36110352 0.36110352]. 
Current patch size: (np.int64(28), np.int64(256), np.int64(256)). 
Current median shape: [ 50.         497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [2.5       

## 🏋️ Train the Model

In [7]:

!nnUNetv2_train 1 3d_fullres 0




############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-07-12 17:36:13.551387: Using torch.compile...
2025-07-12 17:36:15.462798: do_dummy_2d_data_aug: True
2025-07-12 17:36:15.468925: Using splits from existing split file: /content/drive/MyDrive/nnunet_data/nnUNet_preprocessed/Dataset001_AorticValve/splits_final.json
2025-07-12 17:36:16.108842:

In [9]:
##Predicao Casos teste

!nnUNetv2_predict \
-i /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/imagesTs \
-o /content/drive/MyDrive/nnunet_data/predictionsTs \
-d 001 \
-c 3d_fullres \
-f 0 \
-chk checkpoint_best.pth






#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 20 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 20 cases that I would like to predict

Predicting AorticValve_081:
perform_everything_on_device: True
100% 27/27 [00:25<00:00,  1.08it/s]
sending off prediction to background worker for resampling and export
done with AorticValve_081

Predicting AorticValve_082:
perform_everything_on_device: True
100% 64/64 [00:55<00:00,  1.15it/s]
sending off prediction to background worker for resampling and export
done with AorticValve_082

Predicting AorticValve_083:
perform_ever

In [10]:
##Predicao Casos treino
!nnUNetv2_predict \
-d 001 \
-i /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/imagesTr \
-o /content/drive/MyDrive/nnunet_data/predictionsTr \
-c 3d_fullres \
-f 0 \
-chk checkpoint_best.pth



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 80 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 80 cases that I would like to predict

Predicting AorticValve_001:
perform_everything_on_device: True
100% 100/100 [01:31<00:00,  1.09it/s]
sending off prediction to background worker for resampling and export
done with AorticValve_001

Predicting AorticValve_002:
perform_everything_on_device: True
100% 64/64 [00:56<00:00,  1.13it/s]
sending off prediction to background worker for resampling and export
done with AorticValve_002

Predicting AorticValve_003:
perform_ev

In [13]:
#Avaliar performance prediçõs teste
!nnUNetv2_evaluate_folder \
  /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/labelsTs \
  /content/drive/MyDrive/nnunet_data/predictionsTs \
  -djfile /content/drive/MyDrive/nnunet_data/nnUNet_raw/Dataset001_AorticValve/dataset.json \
  -pfile /content/drive/MyDrive/nnunet_data/nnUNet_results/Dataset001_AorticValve/nnUNetTrainer__nnUNetPlans__3d_fullres/plans.json \
  -o /content/drive/MyDrive/mnunet_data/evaluation_results/test_results.json



Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_evaluate_folder", line 8, in <module>
    sys.exit(evaluate_folder_entry_point())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/evaluation/evaluate_predictions.py", line 230, in evaluate_folder_entry_point
    compute_metrics_on_folder2(args.gt_folder, args.pred_folder, args.djfile, args.pfile, args.o, args.np, chill=args.chill)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/evaluation/evaluate_predictions.py", line 194, in compute_metrics_on_folder2
    compute_metrics_on_folder(folder_ref, folder_pred, output_file, rw, file_ending,
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/evaluation/evaluate_predictions.py", line 137, in compute_metrics_on_folder
    assert all(present), "Not all files in folder_ref exist in folder_pred"
           ^^^^^^^^^^^^
AssertionE

In [14]:
##Rodar resultados

import os
import nibabel as nib
import numpy as np
from medpy.metric import binary
import pandas as pd

# Diretórios
labels_dir = '/content/drive/MyDrive/nnUNet/nnUNet_raw/Dataset001_AorticValve/labelsTs'
preds_dir = '/content/drive/MyDrive/nnUNet/predictionsTs'

# Listar arquivos
labels_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.nii.gz')])
preds_files = sorted([f for f in os.listdir(preds_dir) if f.endswith('.nii.gz')])

# Checagem
assert len(labels_files) == len(preds_files), "Número de labels e predições não bate!"

# Inicializar lista de resultados
results = []

# Loop pelos arquivos
for label_file, pred_file in zip(labels_files, preds_files):
    label_path = os.path.join(labels_dir, label_file)
    pred_path = os.path.join(preds_dir, pred_file)

    # Carregar os volumes
    label = nib.load(label_path).get_fdata() > 0
    pred = nib.load(pred_path).get_fdata() > 0

    # Calcular métricas
    dice = binary.dc(pred, label)
    hd95 = binary.hd95(pred, label)

    results.append({
        'Case': label_file.replace('.nii.gz', ''),
        'Dice': round(dice, 4),
        'HD95': round(hd95, 2)
    })

    print(f'{label_file}: Dice={dice:.4f}, HD95={hd95:.2f}')

# Salvar resultados
df = pd.DataFrame(results)
output_path = os.path.join(preds_dir, 'metrics_summary_manual.csv')
df.to_csv(output_path, index=False)

print(f'\nMétricas salvas em {output_path}')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/nnUNet/nnUNet_raw/Dataset001_AorticValve/labelsTs'

In [15]:
##Rodar resultados
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 📄 Ler o CSV gerado
csv_path = '/content/drive/MyDrive/nnUNet/predictionsTs/metrics_summary_manual.csv'
df = pd.read_csv(csv_path)

# ✔️ Mostrar a tabela
print(df)

# 🎯 Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(df.describe())

# 📦 Boxplot Dice
plt.figure(figsize=(6, 6))
sns.boxplot(y=df['Dice'])
plt.title('Dice Coefficient')
plt.grid(True, axis='y')
plt.show()

# 📦 Boxplot HD95
plt.figure(figsize=(6, 6))
sns.boxplot(y=df['HD95'])
plt.title('HD95 (Hausdorff Distance 95)')
plt.grid(True, axis='y')
plt.show()

# 📈 Histograma Dice
plt.figure(figsize=(8, 6))
sns.histplot(df['Dice'], bins=10, kde=True)
plt.title('Dice Distribution')
plt.xlabel('Dice')
plt.ylabel('Count')
plt.grid(True)
plt.show()

# 📈 Histograma HD95
plt.figure(figsize=(8, 6))
sns.histplot(df['HD95'], bins=10, kde=True)
plt.title('HD95 Distribution')
plt.xlabel('HD95 (mm)')
plt.ylabel('Count')
plt.grid(True)
plt.show()

# 📄 Salvar estatísticas descritivas
stats_path = os.path.join(os.path.dirname(csv_path), 'metrics_stats_summary.csv')
df.describe().to_csv(stats_path)
print(f'\nEstatísticas salvas em: {stats_path}')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/nnUNet/predictionsTs/metrics_summary_manual.csv'

In [16]:
##Imprimir imagens - ficaram no plano sagital, pois estavam esticadas no plano axial.

import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Diretórios
images_dir = '/content/drive/MyDrive/nnUNet/nnUNet_raw/Dataset001_AorticValve/imagesTs'
labels_dir = '/content/drive/MyDrive/nnUNet/nnUNet_raw/Dataset001_AorticValve/labelsTs'
preds_dir = '/content/drive/MyDrive/nnUNet/predictionsTs'

# Paciente
patient_id = '087'
image_path = os.path.join(images_dir, f'{patient_id}_0000.nii.gz')
label_path = os.path.join(labels_dir, f'{patient_id}.nii.gz')
pred_path = os.path.join(preds_dir, f'{patient_id}.nii.gz')

# Carregar volumes
img = nib.load(image_path).get_fdata()
label = nib.load(label_path).get_fdata()
pred = nib.load(pred_path).get_fdata()

# Corte sagital (eixo 0)
slice_idx = img.shape[0] // 2
slice_img = img[slice_idx, :, :]
slice_label = label[slice_idx, :, :]
slice_pred = pred[slice_idx, :, :]

# Plot com suavização e tamanho menor
fig, axs = plt.subplots(1, 3, figsize=(9, 3))  # tamanho menor

# CT
axs[0].imshow(slice_img, cmap='gray', origin='lower', interpolation='bilinear')
axs[0].set_title('CT Patch (Sagital)')
axs[0].axis('off')
axs[0].set_aspect('equal')

# Ground Truth
axs[1].imshow(slice_img, cmap='gray', origin='lower', interpolation='bilinear')
axs[1].imshow(slice_label, cmap='Reds', alpha=0.5, origin='lower', interpolation='nearest')
axs[1].set_title('Ground Truth')
axs[1].axis('off')
axs[1].set_aspect('equal')

# Predição
axs[2].imshow(slice_img, cmap='gray', origin='lower', interpolation='bilinear')
axs[2].imshow(slice_pred, cmap='Blues', alpha=0.5, origin='lower', interpolation='nearest')
axs[2].set_title('Prediction')
axs[2].axis('off')
axs[2].set_aspect('equal')

plt.tight_layout()
plt.show()


FileNotFoundError: No such file or no access: '/content/drive/MyDrive/nnUNet/nnUNet_raw/Dataset001_AorticValve/imagesTs/087_0000.nii.gz'